In [2]:
from joblib import dump

from dask.distributed import Client
from dea_tools.classification import predict_xr
import geopandas as gpd
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict

from dep_coastlines.MosaicLoader import MosaicLoader
from dep_tools.namers import DepItemPath
from dep_tools.azure import get_container_client

from dep_coastlines.mask_model import prep_training_data, SavedModel

from lazypredict.Supervised import LazyClassifier


In [9]:
(
    training_data
        .query('count > 4')
        .loc[:,['code', 'red', 'blue', 'green', 'nir08', 'swir16', 'swir22',  'twndwi', 'twndwi_stdev', 'twndwi_mad', 'count']]
        .groupby('code')
        .median()
)

,red,blue,green,nir08,swir16,swir22,twndwi,twndwi_stdev,twndwi_mad,count
code,,,,,,,,,,
bare_terrain,0.04,0.02,0.03,0.11,0.16,0.10,-0.26,0.15,0.06,5.00
clean_water,0.00,0.04,0.04,0.00,0.02,0.02,0.66,0.26,0.15,7.00
cloud,0.14,0.14,0.14,0.16,0.11,0.11,0.01,0.31,0.19,7.00
coast,0.07,0.05,0.07,0.13,0.08,0.05,0.02,0.12,0.07,6.00
deep_ocean,0.01,0.03,0.02,0.01,0.02,0.01,0.55,0.21,0.13,7.00
land,0.04,0.02,0.06,0.41,0.16,0.07,-0.56,0.13,0.03,7.00
noise,0.07,0.14,0.12,0.08,0.10,0.09,0.25,0.38,0.17,6.50
noisy_water,-0.00,0.01,0.01,0.01,0.02,0.02,0.19,0.33,0.13,15.00
surf,0.17,0.18,0.19,0.17,0.05,0.03,0.21,0.30,0.19,8.00


In [22]:

codes = pd.DataFrame.from_records(
   [
        ("land",         1,1,0,1, "#064a00"),
        ('clean_water',  9,2,0,2, "#ff0000"),
        ('water',        2,3,0,2, "#ff0000"),
        ('deep_ocean',   3,3,0,2, "#ff0000"),
        ("noisy_water",  8,7,0,2, "#00ffff"),
        ("surf",         4,4,0,4, "#ddfeff"),
        ("coast",        5,5,0,5, "#ffe48b"),
        ("cloud",        6,6,1,6, "#a9a9a9"),
        ("noise",        7,6,1,6, "#e16be1"),
        ('bare_terrain',10,8,0,8, "#0000ff"),
        ('built',       11,9,0,9, "#ff0000"),         
    ],
    columns=["code", "intcode", "simplintcode", "cloud", "unified_water", "color"],
)

In [86]:
removes = ['shift']#, 'deep_ocean']#, 'deep_ocean', 'water']
training_data = (
    pd.read_csv("../data/training_data_with_features_0-7-0-4_17May2024.csv")
      .query('code not in @removes')
     # .query('count > 4')
     # .dropna()
)

training_data = training_data.join(codes.set_index("code"), on="code")
training_columns = [
        "blue",
        "blue_mad",
        "count",
        "green",
        "green_mad",
        "nir08",
        "nir08_mad",
     #   "nir08_stdev",
        "red",
        "red_mad",
        "swir16",
        "swir16_mad",
        "swir22",
        "swir22_mad",
   #     "blue_dev",
   #     "count_dev",
   #     "green_dev",
   #     "red_dev",
   #     "nir08_dev",
   #     "swir16_dev",
   #     "swir22_dev",
   #    "blue_all",
    #   "count_all",
   #    "green_all",
   #     "nir08_all",
   #     "red_all",
   #    "swir16_all",
   #  "swir22_all",
   # "mndwi",
   # "mndwi_mad",
   # "mndwi_stdev",
   # "ndwi",
   # "ndwi_mad",
   # "ndwi_stdev",
   # "meanwi",
   # "meanwi_mad",
   # "meanwi_stdev",
    "twndwi", 
    "twndwi_mad",
    "twndwi_stdev"
   #  "mndwi_all",
   #  "ndwi_all"
]
training_data = training_data.dropna(subset=training_columns)
X = training_data.loc[:, training_columns].to_numpy()
response_column = 'cloud'
y = training_data[response_column].to_numpy()


In [87]:
m = RandomForestClassifier()
cv = cross_val_score(m, X, y, cv=10)
cxp = cross_val_predict(m, X, y, cv=10)
pd.crosstab(y, cxp)

col_0,0,1
row_0,,
0,1272,146
1,125,633


In [52]:
from scipy.stats import randint

param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)
                }
rand_search = RandomizedSearchCV(m, 
                                 param_distributions = param_dist, 
                                 n_iter=20, 
                                 cv=10)

# Fit the random search object to the data
rand_search.fit(X, y)
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 9, 'n_estimators': 131}


In [82]:
cv.mean()

0.9133644133644132

In [88]:
full_model = m.fit(X, y)

In [ ]:

from sklearn.feature_selection import SelectFromModel
model = SelectFromModel(m, prefit=True)
from itertools import compress
shrunk_columns = list(compress(training_columns,model.get_support()))
X_new = model.transform(X)
cv_shrunk = cross_val_score(m, X_new, y, cv=10)
cxp_shrunk = cross_val_predict(m, X_new, y, cv=10)
pd.crosstab(y, cxp_shrunk)

In [21]:
shrunk_full_model = m.fit(X_new, y)
shrunk_columns

['blue', 'count', 'nir08_mad', 'swir22', 'swir22_mad']

In [89]:

output = SavedModel(
    model = full_model,
    training_data = training_data,
    predictor_columns = training_columns,
    response_column=response_column,
    codes=codes,    
)
dump(output, '../dep_coastlines/full_model_17May2024.joblib')



['../dep_coastlines/full_model_17May2024.joblib']

In [17]:
anitempath = DepItemPath(
        sensor="ls",
        dataset_id="coastlines/mosaics-corrected",
        version="0.6.0",
        time="2022",
        zero_pad_numbers=True,
)
loader = MosaicLoader(itempath=anitempath, add_deviations=True)
cell = (48, 14)

df = pd.DataFrame([cell])
df.index = [cell]
ds = loader.load(df)
ds

<xarray.Dataset>
Dimensions:          (x: 3079, y: 3080)
Coordinates:
    band             int64 1
  * x                (x) float64 6.202e+05 6.202e+05 ... 7.125e+05 7.125e+05
  * y                (y) float64 -2.438e+06 -2.438e+06 ... -2.53e+06 -2.53e+06
    spatial_ref      int64 0
Data variables: (12/57)
    blue             (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    blue_mad         (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    count            (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    green            (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    green_mad        (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    nir08            (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    ...               ...
    swir22_mad_all   (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    mndwi_all        (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    ndwi_all         (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    nirwi_all        (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    meanwi_all       (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>
    maxwi_all        (y, x) float32 dask.array<chunksize=(3080, 3079), meta=np.ndarray>

In [21]:
with Client():
    output = predict_xr(shrunk_full_model, ds[shrunk_columns], clean=True).Predictions

z = ds.nir08.where(output != 1).odc.assign_crs(ds.odc.crs)
output.odc.explore()

predicting...
